## QUERY 3: For every LA Community (COMM) Find Crimes per person & Median Income per person 

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, sum as _sum, avg, round, regexp_replace, trim, format_number

spark = SparkSession.builder \
    .appName("Crime Data Analysis with Median Income") \
    .getOrCreate()

# Load GeoJSON File for 2010 Census Blocks
geojson_path = "s3://initial-notebook-data-bucket-dblab-905418150721/2010_Census_Blocks.geojson"
blocks_df = spark.read.format("geojson") \
    .option("multiLine", "true").load(geojson_path) \
    .selectExpr("explode(features) as features") \
    .select("features.*")

flattened_df = blocks_df.select(
    [col(f"properties.{col_name}").alias(col_name) for col_name in blocks_df.schema["properties"].dataType.fieldNames()] + ["geometry"]
).drop("properties").drop("type")

# Group Polygons per COMM
grouped_blocks_df = flattened_df.groupBy("COMM").agg(
    expr("ST_Union_Aggr(geometry)").alias("unified_geometry")
)

# Load Crime Data
crime_data_path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
crime_df = spark.read.csv(crime_data_path, header=True, inferSchema=True)

crime_df = crime_df.select(
    col("LAT").alias("latitude"),
    col("LON").alias("longitude")
)
crime_df = crime_df.withColumn("geom", expr("ST_Point(longitude, latitude)"))

# Count Crimes per COMM
join_condition = expr("ST_Within(geom, unified_geometry)")
joined_df = crime_df.join(grouped_blocks_df, join_condition, "inner")

crime_counts = joined_df.groupBy("COMM").count()

# Total Population per COMM
community_population_df = flattened_df.groupBy("COMM").agg(
    _sum("POP_2010").alias("total_population")
)

# Total Housing ανά COMM
community_housing_df = flattened_df.groupBy("COMM").agg(
    _sum("HOUSING10").alias("total_housing")
)

# Loan Income Data
income_data = spark.read.option("header", "true").csv("s3://initial-notebook-data-bucket-dblab-905418150721/LA_income_2015.csv")

# Estimated Median Income Column
income_data = income_data.withColumn(
    "Estimated Median Income", 
    regexp_replace(trim(col("Estimated Median Income")), "[$,]", "").cast("float")
)

# Community Column
income_data = income_data.withColumn(
    "COMM", 
    regexp_replace(col("Community"), "Los Angeles \\(|\\)", "").alias("COMM")
)

# Calculate median income per COMM
median_income_per_comm = income_data.groupBy("COMM").agg(
    round(avg("Estimated Median Income"), 2).alias("median_income")
)

# Join crime_counts, community_population_df, community_housing_df και median_income_per_comm
final_df = crime_counts \
    .join(community_population_df, on="COMM", how="inner") \
    .join(community_housing_df, on="COMM", how="inner") \
    .join(median_income_per_comm, on="COMM", how="left")  

# Calculate Crime per person
final_df = final_df.withColumn(
    "crimes_per_person", format_number(col("count") / col("total_population"), 5)
)

# Calculate Median Income per Person
final_df = final_df.withColumn(
    "median_income_per_person", 
    round((col("total_housing") * col("median_income")) / col("total_population"), 2)
)

# Filter NULL or empty COMM
final_df = final_df.filter(col("COMM").isNotNull() & (col("COMM") != ""))

# Sort & Print
final_df = final_df.orderBy("median_income_per_person", ascending=False)

sorted_columns = sorted(final_df.columns)  

final_df.select(sorted_columns).show(n=final_df.count(), truncate=False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------+-----+-----------------+-------------+------------------------+-------------+----------------+
|COMM                          |count|crimes_per_person|median_income|median_income_per_person|total_housing|total_population|
+------------------------------+-----+-----------------+-------------+------------------------+-------------+----------------+
|Malibu                        |1    |0.00008          |123681.0     |67136.92                |6864         |12645           |
|Manhattan Beach               |32   |0.00091          |143527.0     |60985.19                |14929        |35135           |
|Hermosa Beach                 |18   |0.00092          |111187.0     |57924.86                |10162        |19506           |
|Playa Vista                   |4332 |0.48532          |104367.0     |56217.4                 |4808         |8926            |
|La Cañada Flintridge          |1    |0.00005          |156933.0     |54949.03                |7089         |20